# DeepRoof Inference -- Roof Layout + Geometry + 3D.

**Per roof plane:** instance mask, polygon, class (flat/sloped), slope angle, azimuth, normal vector.

**Visualizations:** semantic map, instance color map, normal map (RGB), overlay with slope labels, **interactive 3D roof model**.

In [ ]:
from __future__ import annotations

import json, os, sys, glob
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import torch
import torch.nn.functional as F

In [ ]:
# ======================== CONFIGURATION ========================
MODEL_INPUT_SIZE = 512
MASK_THRESHOLD = 0.55     # Mask sigmoid threshold for roof vs background
SCORE_THRESHOLD = 0.40    # Min combined score to keep a query as an instance
MIN_INSTANCE_AREA = 120   # Min pixels for a valid instance (at 128x128 mask res)
INSTANCE_NMS_IOU = 0.55   # NMS IoU threshold for duplicate query suppression
MAX_INSTANCES = 64        # Safety cap for visualization/export


def detect_project_root() -> Path:
    for c in [Path.cwd(), Path.cwd().parent, Path('/workspace/roof'), Path('/Users/voskan/Desktop/DeepRoof-2026')]:
        if (c / 'configs').exists() and (c / 'deeproof').exists():
            return c
    raise FileNotFoundError('Could not auto-detect project root.')


PROJECT_ROOT = detect_project_root()
CONFIG_PATH = PROJECT_ROOT / 'configs' / 'deeproof_production_swin_L.py'

# Check fine-tune dir first, then scratch
WORK_DIR = PROJECT_ROOT / 'work_dirs' / 'deeproof_absolute_ideal_v1'
if not WORK_DIR.exists():
    WORK_DIR = PROJECT_ROOT / 'work_dirs' / 'swin_l_scratch_v1'

# Prefer best validation checkpoint, fallback to latest iter checkpoint
best_ckpts = sorted(WORK_DIR.glob('best_mIoU*.pth'))
if best_ckpts:
    CHECKPOINT_PATH = best_ckpts[-1]
else:
    last_ckpt_ptr = WORK_DIR / 'last_checkpoint'
    if last_ckpt_ptr.exists():
        t = Path(last_ckpt_ptr.read_text().strip())
        CHECKPOINT_PATH = t if t.is_absolute() else WORK_DIR / t
    else:
        ckpts = sorted(WORK_DIR.glob('iter_*.pth'))
        CHECKPOINT_PATH = ckpts[-1] if ckpts else WORK_DIR / 'iter_16000.pth'

# ---- INPUT IMAGE ----
# Option A: external image
INPUT_IMAGE_PATH = Path('/workspace/test.png')

# Option B: auto-find first OmniCity val image (uncomment next 3 lines)
# val_txt = PROJECT_ROOT / 'data' / 'OmniCity' / 'val.txt'
# first_val = val_txt.read_text().strip().split('\n')[0].strip()
# INPUT_IMAGE_PATH = PROJECT_ROOT / 'data' / 'OmniCity' / 'images' / f'{first_val}.jpg'

if not INPUT_IMAGE_PATH.exists():
    fallback = PROJECT_ROOT / 'test.png'
    if fallback.exists():
        INPUT_IMAGE_PATH = fallback

OUTPUT_DIR = PROJECT_ROOT / 'outputs' / 'checkpoint_inference'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'CHECKPOINT: {CHECKPOINT_PATH}')
print(f'INPUT: {INPUT_IMAGE_PATH}')
print(f'MASK_THRESHOLD: {MASK_THRESHOLD}, SCORE_THRESHOLD: {SCORE_THRESHOLD}')

In [ ]:
# ======================== IMAGE PREPROCESSING ========================
for p in (CONFIG_PATH, CHECKPOINT_PATH, INPUT_IMAGE_PATH):
    if not p.exists():
        raise FileNotFoundError(f'Not found: {p}')

img_orig_bgr = cv2.imread(str(INPUT_IMAGE_PATH), cv2.IMREAD_COLOR)
orig_h, orig_w = img_orig_bgr.shape[:2]

# Center-crop to square, resize to training resolution
crop_size = min(orig_h, orig_w)
y0 = (orig_h - crop_size) // 2
x0 = (orig_w - crop_size) // 2
img_bgr = cv2.resize(img_orig_bgr[y0:y0+crop_size, x0:x0+crop_size],
                      (MODEL_INPUT_SIZE, MODEL_INPUT_SIZE), interpolation=cv2.INTER_AREA)
PREPROCESSED_PATH = OUTPUT_DIR / 'preprocessed.png'
cv2.imwrite(str(PREPROCESSED_PATH), img_bgr)

img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
H, W = MODEL_INPUT_SIZE, MODEL_INPUT_SIZE
print(f'{orig_w}x{orig_h} -> crop {crop_size} -> {H}x{W}')

In [ ]:
# ======================== SLIDING WINDOW CONFIG (SOTA) ========================
# For large satellite images (e.g. 2048x2048), we process in 1024x1024 tiles with overlap.
USE_SLIDING_WINDOW = False  # Set to True for very high-res imagery
TILE_SIZE = 1024
STRIDE = 768  # 256px overlap

if USE_SLIDING_WINDOW:
    print(f'[*] Sliding window active: {TILE_SIZE}x{TILE_SIZE} with stride {STRIDE}')
    # Note: Full implementation would require merging instance results across tiles.
    # This notebook currently optimizes for a single high-fidelity crop.

In [ ]:
# ======================== LOAD MODEL ========================
os.environ.setdefault('TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD', '1')
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from mmseg.utils import register_all_modules
from mmseg.apis import init_model
from mmengine.config import ConfigDict
from mmengine.dataset import Compose

register_all_modules(init_default_scope=False)
import deeproof.models.backbones.swin_v2_compat
import deeproof.models.deeproof_model
import deeproof.models.heads.mask2former_head
import deeproof.models.heads.geometry_head
import deeproof.models.losses

model = init_model(str(CONFIG_PATH), str(CHECKPOINT_PATH), device=DEVICE)
model.test_cfg = ConfigDict(dict(mode='whole'))
model.eval()
print(f'Model loaded. geometry_head: {hasattr(model, "geometry_head")}')

In [ ]:
# ======================== FORWARD PASS ========================
test_pipeline = model.cfg.get('test_pipeline', [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(MODEL_INPUT_SIZE, MODEL_INPUT_SIZE), keep_ratio=False),
    dict(type='PackSegInputs'),
])
pipeline = Compose(test_pipeline)
data = pipeline(dict(img_path=str(PREPROCESSED_PATH)))
data_batch = model.data_preprocessor(
    dict(inputs=[data['inputs']], data_samples=[data['data_samples']]), False
)

# TTA + Feature Extraction
with torch.no_grad():
    # For SOTA results, we can run inference on mirrored/flipped versions and average
    # But for Mask2Former, averaging logits is complex. Better to use ensemble if possible.
    # Here we perform standard high-fidelity forward pass.
    x = model.extract_feat(data_batch['inputs'])
    all_cls_scores, all_mask_preds = model.decode_head(x, data_batch['data_samples'])

    # GeometryHead: surface normals per query
    # FIX: Use model._normalize_query_embeddings() which handles:
    #  - dimension adaptation (cls logits dim=4 padded to embed_dims=256)
    #  - ndim normalization to [B, Q, C]
    # Without this, cls-logit fallback (dim=4) causes shape error in Linear(256,256).
    geo_preds = None
    if hasattr(model, 'geometry_head'):
        raw_qe = getattr(model.decode_head, 'last_query_embeddings', None)
        qe = model._normalize_query_embeddings(
            raw_qe,
            batch_size=1,
            all_cls_scores=all_cls_scores,
        )
        if qe is not None:
            geo_preds = model.geometry_head(qe)  # [1, Q, 3]
        else:
            print('WARNING: query embeddings unavailable — geometry head skipped')

# Last decoder layer
cls_scores = all_cls_scores[-1][0]   # [Q, C+1]
mask_logits = all_mask_preds[-1][0]  # [Q, h, w]
mask_h, mask_w = mask_logits.shape[-2:]

cls_probs = torch.softmax(cls_scores, dim=-1)   # [Q, C+1]
obj_probs = cls_probs[:, :-1]                     # [Q, C] (exclude no-obj)
no_obj_probs = cls_probs[:, -1]                    # [Q]
mask_sigmoid = mask_logits.sigmoid()               # [Q, h, w]

Q = cls_scores.shape[0]
C = obj_probs.shape[1]
print(f'Queries: {Q}, Classes: {C}, Mask res: {mask_h}x{mask_w}')
if geo_preds is not None:
    print(f'GeometryHead: {geo_preds.shape}')

In [ ]:
# ======================== PER-QUERY INSTANCE EXTRACTION ========================
query_normals = geo_preds[0].cpu().numpy() if geo_preds is not None else None  # [Q, 3]

instances = []
for qi in range(Q):
    best_cls = int(obj_probs[qi].argmax())
    # Skip background class predictions early.
    if best_cls <= 0:
        continue

    best_cls_prob = float(obj_probs[qi, best_cls])
    p_no_obj = float(no_obj_probs[qi])

    mask_q = mask_sigmoid[qi]
    binary_mask_lr = (mask_q > MASK_THRESHOLD)
    area_lr = int(binary_mask_lr.sum())

    if area_lr < MIN_INSTANCE_AREA:
        continue

    mean_mask_in_region = float(mask_q[binary_mask_lr].mean())
    combined_score = best_cls_prob * (1.0 - p_no_obj) * mean_mask_in_region
    if combined_score < SCORE_THRESHOLD:
        continue

    mask_full = F.interpolate(
        binary_mask_lr.float().unsqueeze(0).unsqueeze(0),
        size=(H, W), mode='nearest'
    )[0, 0].bool().cpu().numpy()
    area_full = int(mask_full.sum())
    if area_full < MIN_INSTANCE_AREA:
        continue

    normal = None
    slope_deg = None
    azimuth_deg = None
    if query_normals is not None:
        normal = query_normals[qi]
        nz = float(np.clip(abs(normal[2]), -1.0, 1.0))
        slope_deg = float(np.degrees(np.arccos(nz)))
        azimuth_deg = float(np.degrees(np.arctan2(normal[1], normal[0]))) % 360

    class_name = {0: 'background', 1: 'flat_roof', 2: 'sloped_roof'}.get(best_cls, f'cls_{best_cls}')

    instances.append({
        'query_id': qi,
        'mask': mask_full,
        'class_id': best_cls,
        'class_name': class_name,
        'score': round(combined_score, 4),
        'cls_prob': round(best_cls_prob, 4),
        'no_obj_prob': round(p_no_obj, 4),
        'area_px': area_full,
        'normal': normal.tolist() if normal is not None else None,
        'slope_deg': round(slope_deg, 1) if slope_deg is not None else None,
        'azimuth_deg': round(azimuth_deg, 0) if azimuth_deg is not None else None,
    })


def _mask_iou(mask_a, mask_b):
    inter = np.logical_and(mask_a, mask_b).sum()
    if inter == 0:
        return 0.0
    union = np.logical_or(mask_a, mask_b).sum()
    return float(inter) / max(float(union), 1.0)


instances.sort(key=lambda x: x['score'], reverse=True)
instances_before_nms = len(instances)
kept = []
for cand in instances:
    suppress = False
    for prev in kept:
        if cand['class_id'] != prev['class_id']:
            continue
        if _mask_iou(cand['mask'], prev['mask']) >= INSTANCE_NMS_IOU:
            suppress = True
            break
    if not suppress:
        kept.append(cand)
    if len(kept) >= MAX_INSTANCES:
        break

instances = sorted(kept, key=lambda x: x['area_px'], reverse=True)
print(f'NMS: kept {len(instances)} / {instances_before_nms} instances (IoU={INSTANCE_NMS_IOU})')

print(f'\n=== {len(instances)} ROOF PLANE INSTANCES ===')
print(f'{"#":>3} {"Query":>5} {"Class":>12} {"Slope":>8} {"Az":>6} {"Normal":>22} {"Score":>6} {"Area":>7}')
print('-' * 78)
for i, inst in enumerate(instances):
    n_str = f'({inst["normal"][0]:+.2f},{inst["normal"][1]:+.2f},{inst["normal"][2]:+.2f})' if inst['normal'] else 'N/A'
    s_str = f'{inst["slope_deg"]:.0f} deg' if inst['slope_deg'] is not None else 'N/A'
    a_str = f'{inst["azimuth_deg"]:.0f} deg' if inst['azimuth_deg'] is not None else 'N/A'
    print(f'{i:3d} Q{inst["query_id"]:>4d} {inst["class_name"]:>12} {s_str:>8} {a_str:>6} {n_str:>22} {inst["score"]:6.3f} {inst["area_px"]:7d}')


In [ ]:
# ======================== BUILD MAPS ========================
sem_map = np.zeros((H, W), dtype=np.uint8)
inst_map = np.zeros((H, W), dtype=np.int32)
normal_map = np.zeros((H, W, 3), dtype=np.float32)
normal_map[:, :, 2] = 1.0
slope_map = np.zeros((H, W), dtype=np.float32)

for i, inst in enumerate(instances):
    mask = inst['mask']
    sem_map[mask] = inst['class_id']
    inst_map[mask] = i + 1
    if inst['normal'] is not None:
        normal_map[mask] = inst['normal']
    if inst['slope_deg'] is not None:
        slope_map[mask] = inst['slope_deg']

unique_cls = np.unique(sem_map).tolist()
class_areas = {int(c): float((sem_map == c).sum()) / (H * W) for c in unique_cls}
print(f'Semantic classes: {unique_cls}, areas: {class_areas}')
print(f'Roof coverage: {float((sem_map > 0).sum()) / (H * W):.1%}')
print(f'Unique instances: {len(np.unique(inst_map)) - 1}')

In [ ]:
# ======================== VISUALIZATION (4 panels) ========================
sem_palette = np.array([[0,0,0], [0,200,0], [220,50,50]], dtype=np.uint8)
sem_vis = sem_palette[np.clip(sem_map, 0, 2)]

np.random.seed(42)
n_inst = len(instances)
inst_colors = np.random.randint(60, 255, size=(n_inst + 1, 3), dtype=np.uint8)
inst_colors[0] = [0, 0, 0]
inst_vis = inst_colors[np.clip(inst_map, 0, n_inst)]

normal_vis = ((normal_map + 1.0) * 0.5 * 255).clip(0, 255).astype(np.uint8)
normal_vis[sem_map == 0] = 0

fig, axes = plt.subplots(2, 2, figsize=(16, 16))

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Input image')
axes[0, 0].axis('off')

axes[0, 1].imshow(sem_vis)
patches = [
    mpatches.Patch(color=[0,0,0], label='Background'),
    mpatches.Patch(color=np.array([0,200,0])/255, label=f'Flat roof ({class_areas.get(1,0):.0%})'),
    mpatches.Patch(color=np.array([220,50,50])/255, label=f'Sloped roof ({class_areas.get(2,0):.0%})'),
]
axes[0, 1].legend(handles=patches, loc='lower right', fontsize=9)
axes[0, 1].set_title('Semantic map')
axes[0, 1].axis('off')

axes[1, 0].imshow(inst_vis)
axes[1, 0].set_title(f'Instance map ({len(instances)} planes)')
axes[1, 0].axis('off')

axes[1, 1].imshow(normal_vis)
axes[1, 1].set_title('Normal map (R=nx, G=ny, B=nz)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / '4panel_result.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f'Saved: {OUTPUT_DIR / "4panel_result.png"}')

In [ ]:
# ======================== OVERLAY WITH SLOPE LABELS ========================
from deeproof.utils.vectorization import regularize_building_polygons

overlay = cv2.addWeighted(img_rgb, 0.55, sem_vis, 0.45, 0.0)

roof_polygons = []
for i, inst in enumerate(instances):
    reg_polygons = regularize_building_polygons(
        inst['mask'].astype(np.uint8),
        epsilon_factor=0.015,
        ortho_threshold=8.0,
        min_area=50
    )

    for poly in reg_polygons:
        poly_2d = np.asarray(poly).reshape(-1, 2)
        if poly_2d.shape[0] < 3:
            continue

        poly_f32 = np.ascontiguousarray(poly_2d.astype(np.float32).reshape(-1, 1, 2))
        poly_i32 = np.ascontiguousarray(np.rint(poly_2d).astype(np.int32).reshape(-1, 1, 2))

        area = float(cv2.contourArea(poly_f32))
        if area <= 1.0:
            continue

        cv2.polylines(overlay, [poly_i32], True, (255, 255, 255), 1, cv2.LINE_AA)

        m = cv2.moments(poly_f32)
        if m['m00'] > 0:
            cx, cy = int(m['m10'] / m['m00']), int(m['m01'] / m['m00'])
        else:
            cx, cy = int(poly_i32[0, 0, 0]), int(poly_i32[0, 0, 1])

        if inst['slope_deg'] is not None and area > 200:
            txt = f"{inst['slope_deg']:.0f}deg"
            cv2.putText(overlay, txt, (cx - 10, cy + 4), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 255, 0), 1, cv2.LINE_AA)

        roof_polygons.append({
            'polygon_id': len(roof_polygons),
            'instance_id': i,
            'query_id': inst['query_id'],
            'class_id': inst['class_id'],
            'class_name': inst['class_name'],
            'score': inst['score'],
            'area_px': area,
            'slope_deg': inst['slope_deg'],
            'azimuth_deg': inst['azimuth_deg'],
            'normal': inst['normal'],
            'points_xy': poly_i32.reshape(-1, 2).astype(int).tolist(),
        })

cv2.imwrite(str(OUTPUT_DIR / 'overlay.png'), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
cv2.imwrite(str(OUTPUT_DIR / 'semantic_mask.png'), sem_map)

plt.figure(figsize=(12, 12))
plt.imshow(overlay)
plt.title(f'{len(roof_polygons)} regularized roof polygons with slope angles')
plt.axis('off')
plt.show()
print(f'Total polygons: {len(roof_polygons)}')

---
## Interactive 3D Roof Visualization

Each roof facet is rendered as a **3D tilted surface** using the predicted normal vector `(nx, ny, nz)`.

- **Drag** to rotate the view
- **Scroll** to zoom in/out
- **Hover** over any facet to see slope, azimuth, class, and normal vector

In [ ]:
# Ensure plotly is available for interactive 3D
try:
    import plotly
    print(f'plotly {plotly.__version__} ready')
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'plotly'])
    import plotly
    print(f'plotly {plotly.__version__} installed')

# Enable inline rendering for classic Jupyter Notebook
import plotly.offline
plotly.offline.init_notebook_mode(connected=True)
print('Plotly notebook mode initialized -- 3D will render inline below')

In [ ]:
# ======================== INTERACTIVE 3D ROOF MODEL ========================
import plotly.graph_objects as go

# --- 3D Settings ---
Z_SCALE = 4.0          # Height exaggeration (roofs are shallow, need amplification)
WALL_HEIGHT = 10.0     # Building wall height in pixel-space units
ROOF_BASE = WALL_HEIGHT

# Vivid color palette
FACET_COLORS = [
    '#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7',
    '#DDA0DD', '#98D8C8', '#F7DC6F', '#BB8FCE', '#85C1E9',
    '#F0B27A', '#82E0AA', '#F1948A', '#AED6F1', '#D7BDE2',
    '#A3E4D7', '#FAD7A0', '#A9CCE3', '#D5F5E3', '#FADBD8',
]

fig3d = go.Figure()

# --- Dark ground plane ---
fig3d.add_trace(go.Mesh3d(
    x=[0, W, W, 0],
    y=[0, 0, H, H],
    z=[0, 0, 0, 0],
    i=[0, 0], j=[1, 2], k=[2, 3],
    color='#1a1a2e',
    opacity=0.5,
    showlegend=False,
    hoverinfo='skip',
    name='Ground',
))

# --- Render each detected roof facet as 3D ---
for i, inst in enumerate(instances):
    if inst['normal'] is None:
        continue

    nx_n, ny_n, nz_n = inst['normal']
    if abs(nz_n) < 0.01:
        nz_n = 0.01 if nz_n >= 0 else -0.01

    color = FACET_COLORS[i % len(FACET_COLORS)]
    slope_val = f"{inst['slope_deg']:.1f}" if inst['slope_deg'] is not None else '?'
    az_val = f"{inst['azimuth_deg']:.0f}" if inst['azimuth_deg'] is not None else '?'

    try:
        from deeproof.utils.vectorization import regularize_building_polygons
    except ImportError:
        pass

    reg_polygons = regularize_building_polygons(
        inst['mask'].astype(np.uint8), 
        epsilon_factor=0.015, 
        ortho_threshold=8.0, 
        min_area=200
    )

    for poly in reg_polygons:
        poly_2d = poly.reshape(-1, 2).astype(float)
        if poly_2d.shape[0] < 3:
            continue

        xs = poly_2d[:, 0]
        ys = poly_2d[:, 1]
        cx_p, cy_p = xs.mean(), ys.mean()

        # Compute Z for each vertex: z = -(nx*(x-cx) + ny*(y-cy)) / nz
        zs_roof = -(nx_n * (xs - cx_p) + ny_n * (ys - cy_p)) / nz_n
        zs_roof = zs_roof * Z_SCALE + ROOF_BASE

        n_pts = len(xs)
        if n_pts < 3:
            continue

        # Fan triangulation from centroid for filled roof
        all_x = list(xs) + [cx_p]
        all_y = list(ys) + [cy_p]
        z_center = float(np.mean(zs_roof))
        all_z = list(zs_roof) + [z_center]
        center_idx = n_pts

        tri_i, tri_j, tri_k = [], [], []
        for v in range(n_pts):
            v_next = (v + 1) % n_pts
            tri_i.append(center_idx)
            tri_j.append(v)
            tri_k.append(v_next)

        hover = (
            f'<b>Facet #{i}</b><br>'
            f'Class: {inst["class_name"]}<br>'
            f'Slope: {slope_val} deg<br>'
            f'Azimuth: {az_val} deg<br>'
            f'Normal: ({nx_n:.3f}, {ny_n:.3f}, {nz_n:.3f})<br>'
            f'Score: {inst["score"]:.3f}<br>'
            f'Area: {inst["area_px"]} px'
        )

        # --- Roof surface ---
        fig3d.add_trace(go.Mesh3d(
            x=all_x, y=all_y, z=all_z,
            i=tri_i, j=tri_j, k=tri_k,
            color=color,
            opacity=0.9,
            name=f'#{i} {inst["class_name"]} ({slope_val} deg)',
            hovertext=hover,
            hoverinfo='text',
            showlegend=True,
            flatshading=False,
            lighting=dict(ambient=0.6, diffuse=0.7, specular=0.4, roughness=0.3),
            lightposition=dict(x=200, y=200, z=500),
        ))

        # --- Roof edge outline (white lines) ---
        edge_x = list(xs) + [xs[0], None]
        edge_y = list(ys) + [ys[0], None]
        edge_z = list(zs_roof) + [zs_roof[0], None]
        fig3d.add_trace(go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='white', width=4),
            showlegend=False,
            hoverinfo='skip',
        ))

        # --- Building walls (vertical from ground to roof) ---
        for v in range(n_pts):
            v_next = (v + 1) % n_pts
            wx = [xs[v], xs[v_next], xs[v_next], xs[v]]
            wy = [ys[v], ys[v_next], ys[v_next], ys[v]]
            wz = [0, 0, float(zs_roof[v_next]), float(zs_roof[v])]

            fig3d.add_trace(go.Mesh3d(
                x=wx, y=wy, z=wz,
                i=[0, 0], j=[1, 2], k=[2, 3],
                color=color,
                opacity=0.2,
                showlegend=False,
                hoverinfo='skip',
                flatshading=True,
            ))

# --- Scene Layout ---
fig3d.update_layout(
    title=dict(
        text=f'DeepRoof 3D -- {len(instances)} Roof Facets',
        font=dict(size=20, color='white'),
        x=0.5,
    ),
    scene=dict(
        xaxis=dict(title='X (px)', range=[0, W], showgrid=True,
                   gridcolor='#333', backgroundcolor='#111'),
        yaxis=dict(title='Y (px)', range=[H, 0], showgrid=True,
                   gridcolor='#333', backgroundcolor='#111'),
        zaxis=dict(title='Height', range=[-2, ROOF_BASE * 3.5], showgrid=True,
                   gridcolor='#333', backgroundcolor='#111'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.5),
        bgcolor='#0d1117',
        camera=dict(
            eye=dict(x=1.3, y=-1.3, z=1.0),
            up=dict(x=0, y=0, z=1),
        ),
    ),
    paper_bgcolor='#0d1117',
    plot_bgcolor='#0d1117',
    font=dict(color='#e6e6e6', size=12),
    legend=dict(
        bgcolor='rgba(13,17,23,0.8)',
        bordercolor='#444',
        borderwidth=1,
        font=dict(size=11, color='white'),
        title=dict(text='Detected Facets', font=dict(size=13)),
    ),
    width=950,
    height=750,
    margin=dict(l=10, r=10, t=60, b=10),
)

# Save as interactive HTML (always works)
html_path = OUTPUT_DIR / '3d_roof_interactive.html'
fig3d.write_html(str(html_path), include_plotlyjs='cdn')

# Try to save static PNG (requires kaleido)
try:
    fig3d.write_image(str(OUTPUT_DIR / '3d_roof_model.png'), scale=2)
    print(f'Static PNG saved: {OUTPUT_DIR / "3d_roof_model.png"}')
except Exception:
    pass

# Show inline in notebook
fig3d.show()
print(f'\nInteractive 3D saved: {html_path}')
print('Drag to rotate | Scroll to zoom | Hover for details')

In [ ]:
# ======================== TOP QUERY MASKS ========================
query_areas = [(qi, float((mask_sigmoid[qi] > MASK_THRESHOLD).float().sum())) for qi in range(Q)]
query_areas.sort(key=lambda x: x[1], reverse=True)
top_queries = [qa[0] for qa in query_areas[:8] if qa[1] > 0]

if top_queries:
    n_show = len(top_queries)
    cols = 4
    rows = (n_show + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
    axes = np.array(axes).flatten()
    for idx, qi in enumerate(top_queries):
        mask_vis = mask_sigmoid[qi].cpu().numpy()
        cov = float((mask_sigmoid[qi] > MASK_THRESHOLD).float().sum())
        cls = int(obj_probs[qi].argmax())
        conf = float(obj_probs[qi].max())
        n_str = ''
        if query_normals is not None:
            nz = abs(float(query_normals[qi][2]))
            slope = float(np.degrees(np.arccos(np.clip(nz, -1, 1))))
            n_str = f', {slope:.0f} deg'
        axes[idx].imshow(mask_vis, cmap='hot', vmin=0, vmax=1)
        axes[idx].set_title(f'Q{qi} cls={cls} P={conf:.2f}{n_str}\narea={cov:.0f}px', fontsize=9)
        axes[idx].axis('off')
    for idx in range(len(top_queries), len(axes)):
        axes[idx].axis('off')
    plt.suptitle('Top query masks (sigmoid, hot colormap)', fontsize=13)
    plt.tight_layout()
    plt.show()

In [ ]:
# ======================== SAVE JSON + GEOJSON + OBJ ========================
json_polygons = [{k: v for k, v in p.items()} for p in roof_polygons]

with (OUTPUT_DIR / 'roof_polygons.json').open('w') as f:
    json.dump({
        'image': str(INPUT_IMAGE_PATH),
        'checkpoint': str(CHECKPOINT_PATH),
        'model_input_size': MODEL_INPUT_SIZE,
        'mask_threshold': MASK_THRESHOLD,
        'total_instances': len(instances),
        'total_polygons': len(roof_polygons),
        'polygons': json_polygons,
    }, f, indent=2)

# GeoJSON (pixel-space)
features = []
for p in roof_polygons:
    coords = [[float(np.clip(x, 0, W - 1)), float(np.clip(y, 0, H - 1))] for x, y in p['points_xy']]
    if coords and coords[0] != coords[-1]:
        coords.append(coords[0])
    features.append({
        'type': 'Feature',
        'properties': {k: v for k, v in p.items() if k != 'points_xy'},
        'geometry': {'type': 'Polygon', 'coordinates': [coords]},
    })

with (OUTPUT_DIR / 'roof_polygons.geojson').open('w') as f:
    json.dump({'type': 'FeatureCollection', 'features': features}, f, indent=2)

# OBJ Export (3D mesh for Blender/MeshLab)
WALL_H_OBJ = 10.0
obj_path = OUTPUT_DIR / 'roof_model.obj'
with open(str(obj_path), 'w') as f:
    f.write('# DeepRoof 3D Roof Model\n')
    f.write(f'# {len(instances)} roof facets\n\n')
    vert_offset = 1
    for i, inst in enumerate(instances):
        if inst['normal'] is None:
            continue
        nx_n, ny_n, nz_n = inst['normal']
        if abs(nz_n) < 0.01:
            nz_n = 0.01
        try:
            from deeproof.utils.vectorization import regularize_building_polygons
        except ImportError:
            pass

        reg_polygons = regularize_building_polygons(
            inst['mask'].astype(np.uint8), 
            epsilon_factor=0.015, 
            ortho_threshold=8.0, 
            min_area=100
        )
        for poly in reg_polygons:
            poly_2d = poly.reshape(-1, 2)
            if poly_2d.shape[0] < 3:
                continue
            cx_o = poly_2d[:, 0].mean()
            cy_o = poly_2d[:, 1].mean()
            xs_o = poly_2d[:, 0].astype(float)
            ys_o = poly_2d[:, 1].astype(float)
            zs_o = -(nx_n * (xs_o - cx_o) + ny_n * (ys_o - cy_o)) / nz_n + WALL_H_OBJ
            f.write(f'# Facet {i} ({inst["class_name"]}) slope={inst["slope_deg"]}deg\n')
            f.write(f'vn {nx_n:.4f} {ny_n:.4f} {nz_n:.4f}\n')
            for x, y, z in zip(xs_o, ys_o, zs_o):
                f.write(f'v {x:.2f} {y:.2f} {z:.4f}\n')
            n_verts = len(xs_o)
            face_indices = ' '.join(str(vert_offset + j) for j in range(n_verts))
            f.write(f'f {face_indices}\n\n')
            vert_offset += n_verts

# Summary
summary = {
    'checkpoint': str(CHECKPOINT_PATH),
    'input': str(INPUT_IMAGE_PATH),
    'instances': len(instances),
    'polygons': len(roof_polygons),
    'classes': unique_cls,
    'roof_coverage': round(float((sem_map > 0).sum()) / (H * W), 4),
}
print(json.dumps(summary, indent=2))
print(f'\nAll outputs saved to: {OUTPUT_DIR}')
print(f'3D OBJ file: {obj_path}')
print(f'Interactive HTML: {OUTPUT_DIR / "3d_roof_interactive.html"}')